In [1]:
# basic import statements
import os
import re
import gc
import warnings
import numpy as np
import pandas as pd
import collections
warnings.filterwarnings("ignore")
from datetime import date 

#set max display in notebook
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Show multiple output in console
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Honda Data

In [2]:
df1 = pd.read_csv('honda_sell_data.csv')
df1.shape
df1.head()

(4999, 25)

,Year,Make,Model,Condition,Price,Consumer_Rating,Consumer_Review_#,Exterior_Color,Interior_Color,Drivetrain,MPG,Fuel_Type,Transmission,Engine,VIN,Stock_#,Mileage,Comfort_Rating,Interior_Design_Rating,Performance_Rating,Value_For_Money_Rating,Exterior_Styling_Rating,Reliability_Rating,State,Seller_Type
0,2023,Honda,Ridgeline RTL,New,"$46,370",4.8,9,Platinum White Pearl,Beige,All-wheel Drive,NaN,Gasoline,Automatic,3.5L V6 24V GDI SOHC,5FPYK3F58PB011817,00830164,10,5.0,4.8,4.8,4.2,5.0,5.0,CA,Dealer
1,2023,Honda,CR-V Hybrid Sport,New,"$34,150",1.7,24,Platinum White Pearl,Black,FWD,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS5H5XPH702953,6402953,–,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer
2,2023,Honda,CR-V Hybrid Sport,New,"$34,245",4.7,2869,Meteorite Gray Metallic,Black,Front-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS5H55PH704805,PH704805,1,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer
3,2022,Honda,Pilot TrailSport,New,"$46,500",5.0,12,Sonic Gray Pearl,Black,All-wheel Drive,19–25,Gasoline,9-Speed Automatic,3.5L V6 24V GDI SOHC,5FNYF6H82NB100429,5700429B,5,5.0,5.0,5.0,5.0,5.0,5.0,CA,Dealer
4,2023,Honda,CR-V Hybrid Sport Touring,New,"$40,395",4.4,12,Crystal Black Pearl,Black,All-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS6H90PH805954,5705954,5,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer


In [3]:
df1.Condition.value_counts()

New                2697
Used               2022
Honda Certified     280
Name: Condition, dtype: int64

In [4]:
df1.Model.value_counts()

CR-V EX-L                                             268
Pilot Sport                                           228
Pilot EX-L                                            204
Accord Sport 1.5T                                     195
CR-V Hybrid Sport Touring                             185
CR-V EX                                               178
Odyssey EX-L                                          174
Civic Sport                                           160
Pilot TrailSport                                      150
Ridgeline RTL-E                                       144
Pilot Special Edition                                 136
Passport EX-L                                         128
Ridgeline RTL                                         119
Odyssey Elite                                         109
Accord Hybrid Sport                                   100
Civic Si Base                                          98
Passport Elite                                         97
Civic EX      

In [5]:
df1 = df1[df1.Condition=='New']
df1.shape
df1.head()

(2697, 25)

,Year,Make,Model,Condition,Price,Consumer_Rating,Consumer_Review_#,Exterior_Color,Interior_Color,Drivetrain,MPG,Fuel_Type,Transmission,Engine,VIN,Stock_#,Mileage,Comfort_Rating,Interior_Design_Rating,Performance_Rating,Value_For_Money_Rating,Exterior_Styling_Rating,Reliability_Rating,State,Seller_Type
0,2023,Honda,Ridgeline RTL,New,"$46,370",4.8,9,Platinum White Pearl,Beige,All-wheel Drive,NaN,Gasoline,Automatic,3.5L V6 24V GDI SOHC,5FPYK3F58PB011817,00830164,10,5.0,4.8,4.8,4.2,5.0,5.0,CA,Dealer
1,2023,Honda,CR-V Hybrid Sport,New,"$34,150",1.7,24,Platinum White Pearl,Black,FWD,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS5H5XPH702953,6402953,–,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer
2,2023,Honda,CR-V Hybrid Sport,New,"$34,245",4.7,2869,Meteorite Gray Metallic,Black,Front-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS5H55PH704805,PH704805,1,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer
3,2022,Honda,Pilot TrailSport,New,"$46,500",5.0,12,Sonic Gray Pearl,Black,All-wheel Drive,19–25,Gasoline,9-Speed Automatic,3.5L V6 24V GDI SOHC,5FNYF6H82NB100429,5700429B,5,5.0,5.0,5.0,5.0,5.0,5.0,CA,Dealer
4,2023,Honda,CR-V Hybrid Sport Touring,New,"$40,395",4.4,12,Crystal Black Pearl,Black,All-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS6H90PH805954,5705954,5,5.0,3.0,4.0,4.0,5.0,5.0,CA,Dealer


In [7]:
df1.Seller_Type.value_counts()

Dealer    2690
Name: Seller_Type, dtype: int64

In [10]:
df1 = df1.drop(columns=['Condition','Consumer_Review_#','VIN','Stock_#','Seller_Type'])
df1.shape
df1.head()

(2697, 20)

,Year,Make,Model,Price,Consumer_Rating,Exterior_Color,Interior_Color,Drivetrain,MPG,Fuel_Type,Transmission,Engine,Mileage,Comfort_Rating,Interior_Design_Rating,Performance_Rating,Value_For_Money_Rating,Exterior_Styling_Rating,Reliability_Rating,State
0,2023,Honda,Ridgeline RTL,"$46,370",4.8,Platinum White Pearl,Beige,All-wheel Drive,NaN,Gasoline,Automatic,3.5L V6 24V GDI SOHC,10,5.0,4.8,4.8,4.2,5.0,5.0,CA
1,2023,Honda,CR-V Hybrid Sport,"$34,150",1.7,Platinum White Pearl,Black,FWD,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,–,5.0,3.0,4.0,4.0,5.0,5.0,CA
2,2023,Honda,CR-V Hybrid Sport,"$34,245",4.7,Meteorite Gray Metallic,Black,Front-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,1,5.0,3.0,4.0,4.0,5.0,5.0,CA
3,2022,Honda,Pilot TrailSport,"$46,500",5.0,Sonic Gray Pearl,Black,All-wheel Drive,19–25,Gasoline,9-Speed Automatic,3.5L V6 24V GDI SOHC,5,5.0,5.0,5.0,5.0,5.0,5.0,CA
4,2023,Honda,CR-V Hybrid Sport Touring,"$40,395",4.4,Crystal Black Pearl,Black,All-wheel Drive,NaN,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,5,5.0,3.0,4.0,4.0,5.0,5.0,CA


In [58]:
df1.to_excel('honda_data.xlsx',index=False)

In [62]:
for col in df1.columns:
    print(col)

Year
Make
Model
Price
Consumer_Rating
Exterior_Color
Interior_Color
Drivetrain
MPG
Fuel_Type
Transmission
Engine
Mileage
Comfort_Rating
Interior_Design_Rating
Performance_Rating
Value_For_Money_Rating
Exterior_Styling_Rating
Reliability_Rating
State


In [65]:
df1.Model.value_counts()

Pilot Sport                     217
CR-V EX-L                       189
CR-V Hybrid Sport Touring       185
Accord Sport 1.5T               158
Pilot TrailSport                145
CR-V EX                         107
Ridgeline RTL-E                 106
Pilot Special Edition           104
Accord Hybrid Sport              86
Ridgeline RTL                    85
Pilot EX-L                       83
Civic Sport                      82
HR-V Sport                       74
Passport TrailSport              69
Odyssey Elite                    62
Pilot Elite                      59
Pilot Touring                    54
Accord Sport 2.0T                54
CR-V Hybrid Sport                54
Odyssey Touring                  53
Passport Elite                   53
Ridgeline Black Edition          52
Passport EX-L                    50
Odyssey EX-L                     50
Civic Si Base                    45
Odyssey Sport                    42
HR-V EX-L                        39
Pilot EX-L 8-Passenger      

### Honda Car Sales Data

In [20]:
df2 = pd.read_csv('car_sales_data.csv')
df2.shape
df2.head()

(2500000, 9)

,Date,Salesperson,Customer Name,Car Make,Car Model,Car Year,Sale Price,Commission Rate,Commission Earned
0,2022-08-01,Monica Moore MD,Mary Butler,Nissan,Altima,2018,15983,0.070495,1126.73
1,2023-03-15,Roberto Rose,Richard Pierce,Nissan,F-150,2016,38474,0.134439,5172.40
2,2023-04-29,Ashley Ramos,Sandra Moore,Ford,Civic,2016,33340,0.114536,3818.63
3,2022-09-04,Patrick Harris,Johnny Scott,Ford,Altima,2013,41937,0.092191,3866.20
4,2022-06-16,Eric Lopez,Vanessa Jones,Honda,Silverado,2022,20256,0.113490,2298.85


In [21]:
for col in df2.columns:
    print(col)

Date
Salesperson
Customer Name
Car Make
Car Model
Car Year
Sale Price
Commission Rate
Commission Earned


In [22]:
df2 = df2.rename(columns={"Customer Name": "Customer_Name", "Car Make": "Car_Make","Car Model":"Car_Model","Car Year":"Car_year","Sale Price":"Sale_Price","Commission Rate":"Commission_Rate","Commission Earned":"Commission_Earned"})
df2.shape
df2.head()

(2500000, 9)

,Date,Salesperson,Customer_Name,Car_Make,Car_Model,Car_year,Sale_Price,Commission_Rate,Commission_Earned
0,2022-08-01,Monica Moore MD,Mary Butler,Nissan,Altima,2018,15983,0.070495,1126.73
1,2023-03-15,Roberto Rose,Richard Pierce,Nissan,F-150,2016,38474,0.134439,5172.40
2,2023-04-29,Ashley Ramos,Sandra Moore,Ford,Civic,2016,33340,0.114536,3818.63
3,2022-09-04,Patrick Harris,Johnny Scott,Ford,Altima,2013,41937,0.092191,3866.20
4,2022-06-16,Eric Lopez,Vanessa Jones,Honda,Silverado,2022,20256,0.113490,2298.85


In [24]:
df2 = df2[df2.Car_Make == 'Honda']
df2.shape
df2.head()

(500687, 9)

,Date,Salesperson,Customer_Name,Car_Make,Car_Model,Car_year,Sale_Price,Commission_Rate,Commission_Earned
4,2022-06-16,Eric Lopez,Vanessa Jones,Honda,Silverado,2022,20256,0.113490,2298.85
6,2022-06-12,Ashley Brown,Tyler Lawson,Honda,F-150,2013,41397,0.142780,5910.67
10,2023-03-12,Harold Nelson,Isaac Patton,Honda,Silverado,2021,41259,0.092541,3818.16
15,2022-12-21,James Velasquez,Natalie Thompson,Honda,Corolla,2016,36607,0.055251,2022.57
25,2023-02-15,Faith Pierce,Jimmy Dunn,Honda,Silverado,2016,35279,0.063400,2236.68


In [59]:
df2.to_excel('honda_sales_data.xlsx',index=False)

### Sentiment analysis from Car Reviews

In [27]:
df3 = pd.read_excel('Scrapped_Car_Reviews_Honda.xlsx')
df3.shape
df3.head()

(14213, 7)

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,4 years with our element. Honestly there isn'...,4.0
1,1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,I have owned my Element since 2007 when I pur...,5.0
2,2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,Lowest maintenance of any vehicle I've ever o...,1.0
3,3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",The only thing I'd change is the drink holder...,5.0
4,4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,I bought this vehicle new in 2007. got 97K mi...,4.0


In [28]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kothaapuroop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
# Remove unwanted columns
df3 = df3.drop(columns=['Unnamed: 0'])

In [32]:
df3[df3.isna().any(axis=1)]

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
585,on 09/04/14 09:21 AM (PDT),rsxing,2014 Honda Accord Hybrid Sedan Touring 4dr Sed...,2014 Accord Hybrid Touring MPG WTF,I purchased 2 Honda Accord Hybrid Touring sed...,NaN
586,4.25,NaN,NaN,NaN,NaN,NaN
621,on 03/04/04 00:00 AM (PST),Tim Guiles,2003 Honda Insight Hatchback 2dr Hatchback (1....,It doesn't get any better than this!,In the summer I get 85 miles to the gallon,NaN
622,NaN,NaN,NaN,NaN,NaN,NaN
623,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
12843,NaN,NaN,NaN,NaN,NaN,NaN
12844,3.625,NaN,NaN,NaN,NaN,NaN
13028,on 02/24/16 12:14 PM (PST),Lee,2011 Honda Pilot SUV LX 4dr SUV (3.5L 6cyl 5A),NaN,,4.0
13459,on 09/02/14 23:32 PM (PDT),dabizmike,2014 Honda Pilot SUV EX 4dr SUV 4WD (3.5L 6cyl...,Not a 4WD,Bought this after salesman said this was 4WD ...,NaN


In [34]:
# Handle missing data
df3.dropna(inplace=True)

In [35]:
# Text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

In [36]:
df3['Review'] = df3['Review'].apply(preprocess_text)
df3.shape
df3.head()

(12382, 6)

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0


In [37]:
#Analysis

from textblob import TextBlob

# Perform sentiment analysis and store the results in a new column
df3['Sentiment'] = df3['Review'].apply(lambda x: TextBlob(x).sentiment.polarity)
df3.shape
df3.head()

(12382, 7)

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0,0.142614
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0,0.318987
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0,0.361905
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0,0.169356
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0,-0.019043


In [39]:
average_sentiment = df3['Sentiment'].mean()
print(f'Average Sentiment Score: {average_sentiment}')

Average Sentiment Score: 0.208097428514468


#### Positive and Negative Reviews

In [40]:
# Define a threshold for positive and negative sentiment
threshold = 0.0

# Create a new column to categorize reviews
df3['Sentiment_Category'] = df3['Sentiment'].apply(lambda x: 'Positive' if x > threshold else 'Negative' if x < -threshold else 'Neutral')

In [41]:
df3.shape
df3.head()

(12382, 8)

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Sentiment_Category
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0,0.142614,Positive
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0,0.318987,Positive
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0,0.361905,Positive
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0,0.169356,Positive
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0,-0.019043,Negative


#### Tokenize and count words

In [43]:
from collections import Counter
from nltk.tokenize import word_tokenize

# Tokenize the reviews
df3['Tokenized_Review'] = df3['Review'].apply(word_tokenize)

In [44]:
df3.shape

(12382, 9)

In [45]:
df3.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Sentiment_Category,Tokenized_Review
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0,0.142614,Positive,"[years, element, honestly, much, complain, nev..."
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0,0.318987,Positive,"[owned, element, since, purchased, new, great,..."
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0,0.361905,Positive,"[lowest, maintenance, vehicle, ever, owned, ti..."
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0,0.169356,Positive,"[thing, change, drink, holders, hard, get, arm..."
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0,-0.019043,Negative,"[bought, vehicle, new, got, k, miles, served, ..."


In [47]:
# Separate positive and negative reviews
positive_reviews = df3[df3['Sentiment_Category'] == 'Positive']
negative_reviews = df3[df3['Sentiment_Category'] == 'Negative']

# Count words in positive and negative reviews
positive_word_counts = Counter()
negative_word_counts = Counter()

for review in positive_reviews['Tokenized_Review']:
    positive_word_counts.update(review)

for review in negative_reviews['Tokenized_Review']:
    negative_word_counts.update(review)

In [48]:
# Get the most frequent words in positive reviews
most_common_positive_words = positive_word_counts.most_common(10)

# Get the most frequent words in negative reviews
most_common_negative_words = negative_word_counts.most_common(10)

print("Most Common Words in Positive Reviews:")
for word, count in most_common_positive_words:
    print(f"{word}: {count}")

print("\nMost Common Words in Negative Reviews:")
for word, count in most_common_negative_words:
    print(f"{word}: {count}")

Most Common Words in Positive Reviews:
car: 16187
x: 9792
honda: 7539
great: 5474
miles: 5309
like: 3788
drive: 3759
mpg: 3562
good: 3423
get: 3420

Most Common Words in Negative Reviews:
car: 2146
honda: 1846
x: 1198
miles: 851
k: 489
get: 480
bought: 478
transmission: 466
like: 452
new: 441


#### Understanding the context of the reviews 

In [49]:
from nltk.util import ngrams

# Define a function to extract n-grams from a list of tokens
def get_ngrams(tokens, n):
    return list(ngrams(tokens, n))

In [50]:
# Tokenize the reviews
df3['Tokenized_Review'] = df3['Review'].apply(word_tokenize)
df3.shape
df3.head()

(12382, 9)

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Sentiment_Category,Tokenized_Review
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0,0.142614,Positive,"[years, element, honestly, much, complain, nev..."
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0,0.318987,Positive,"[owned, element, since, purchased, new, great,..."
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0,0.361905,Positive,"[lowest, maintenance, vehicle, ever, owned, ti..."
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0,0.169356,Positive,"[thing, change, drink, holders, hard, get, arm..."
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0,-0.019043,Negative,"[bought, vehicle, new, got, k, miles, served, ..."


In [53]:
# Extract and count bi-grams or trigrams in positive and negative reviews
positive_reviews['Bi-grams'] = positive_reviews['Tokenized_Review'].apply(lambda x: get_ngrams(x, 3))  # Change 2 to 3 for trigrams
negative_reviews['Bi-grams'] = negative_reviews['Tokenized_Review'].apply(lambda x: get_ngrams(x, 3))  # Change 2 to 3 for trigrams

In [54]:
# Count the frequency of bi-grams or trigrams
positive_bi_gram_counts = Counter()
negative_bi_gram_counts = Counter()

for bi_grams in positive_reviews['Bi-grams']:
    positive_bi_gram_counts.update(bi_grams)

for bi_grams in negative_reviews['Bi-grams']:
    negative_bi_gram_counts.update(bi_grams)

In [55]:
positive_bi_gram_counts

Counter({('years', 'element'): 1,
         ('element', 'honestly'): 2,
         ('honestly', 'much'): 1,
         ('much', 'complain'): 2,
         ('complain', 'never'): 1,
         ('never', 'grew'): 2,
         ('grew', 'love'): 1,
         ('love', 'transmission'): 2,
         ('transmission', 'annoys'): 1,
         ('annoys', 'constantly'): 1,
         ('constantly', 'hunting'): 3,
         ('hunting', 'thru'): 1,
         ('thru', 'gear'): 1,
         ('gear', 'yes'): 1,
         ('yes', 'know'): 2,
         ('know', 'getting'): 8,
         ('getting', 'best'): 7,
         ('best', 'mileage'): 21,
         ('mileage', 'shifts'): 1,
         ('shifts', 'hard'): 3,
         ('hard', 'harsh'): 1,
         ('harsh', 'fact'): 1,
         ('fact', 'unpleasant'): 1,
         ('unpleasant', 'find'): 1,
         ('find', 'annoying'): 4,
         ('annoying', 'listening'): 1,
         ('listening', 'engine'): 1,
         ('engine', 'cycle'): 1,
         ('cycle', 'trans'): 1,
         ('tr

#### Get the most frequest bigrams or trigrams 

In [57]:
# Get the most frequent bi-grams or trigrams in positive reviews
most_common_positive_bi_grams = positive_bi_gram_counts.most_common(30)  # Change 10 to your desired number
# Get the most frequent bi-grams or trigrams in negative reviews
most_common_negative_bi_grams = negative_bi_gram_counts.most_common(30)  # Change 10 to your desired number

print("Most Common Bi-grams/Trigrams in Positive Reviews:")
for bi_gram, count in most_common_positive_bi_grams:
    print(f"{' '.join(bi_gram)}: {count}")

print("\nMost Common Bi-grams/Trigrams in Negative Reviews:")
for bi_gram, count in most_common_negative_bi_grams:
    print(f"{' '.join(bi_gram)}: {count}")

Most Common Bi-grams/Trigrams in Positive Reviews:
gas mileage: 1454
k miles: 1186
fun drive: 897
great car: 735
bought car: 657
cr v: 576
fuel economy: 515
love car: 503
road noise: 499
honda civic: 407
car x: 391
honda accord: 378
ex l: 378
brand new: 372
new car: 359
great gas: 339
get mpg: 330
ever owned: 304
car great: 294
car ever: 283
x car: 280
oil changes: 272
around town: 271
best car: 262
much better: 258
steering wheel: 247
sound system: 239
back seat: 236
buy another: 234
average mpg: 233

Most Common Bi-grams/Trigrams in Negative Reviews:
buy another honda: 29
check engine light: 23
never buy another: 22
never buy honda: 18
car ever owned: 16
miles per gallon: 16
honda cr v: 14
buy another one: 13
power steering pump: 13
bought car new: 12
good gas mileage: 11
honda customer service: 11
would buy another: 10
class action lawsuit: 10
worst car ever: 10
brand new car: 10
tu tu tu: 10
honda stand behind: 9
first last honda: 9
car k miles: 9
better gas mileage: 9
bought car u

In [66]:
df_most_common_positive = pd.DataFrame(most_common_positive_bi_grams, columns=['Bi-gram/Trigram', 'Count'])
df_most_common_negative = pd.DataFrame(most_common_negative_bi_grams, columns=['Bi-gram/Trigram', 'Count'])

In [67]:
df_most_common_positive

,Bi-gram/Trigram,Count
0,"(gas, mileage)",1454
1,"(k, miles)",1186
2,"(fun, drive)",897
3,"(great, car)",735
4,"(bought, car)",657
5,"(cr, v)",576
6,"(fuel, economy)",515
7,"(love, car)",503
8,"(road, noise)",499
9,"(honda, civic)",407


In [68]:
df_most_common_negative

,Bi-gram/Trigram,Count
0,"(buy, another, honda)",29
1,"(check, engine, light)",23
2,"(never, buy, another)",22
3,"(never, buy, honda)",18
4,"(car, ever, owned)",16
5,"(miles, per, gallon)",16
6,"(honda, cr, v)",14
7,"(buy, another, one)",13
8,"(power, steering, pump)",13
9,"(bought, car, new)",12


In [69]:
df_most_common_positive.to_excel('common_positive.xlsx',index=False)
df_most_common_negative.to_excel('common_negative.xlsx',index=False)

In [60]:
df3.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment,Sentiment_Category,Tokenized_Review
0,on 03/01/17 13:11 PM (PST),kris corah,2007 Honda Element SUV LX 4dr SUV AWD (2.4L 4c...,QUIRKY AND PRACTICAL,years element honestly much complain never gre...,4.0,0.142614,Positive,"[years, element, honestly, much, complain, nev..."
1,on 09/02/16 18:22 PM (PDT),Carol Shafford,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Go Anywhere with your Element,owned element since purchased new great little...,5.0,0.318987,Positive,"[owned, element, since, purchased, new, great,..."
2,on 11/12/15 08:04 AM (PST),Gale,2007 Honda Element SUV EX 4dr SUV AWD (2.4L 4c...,Such a cool car!,lowest maintenance vehicle ever owned time mov...,1.0,0.361905,Positive,"[lowest, maintenance, vehicle, ever, owned, ti..."
3,on 08/13/15 15:40 PM (PDT),Judy Eastham,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5M),"From soup to nuts, it does it all.",thing change drink holders hard get arm love c...,5.0,0.169356,Positive,"[thing, change, drink, holders, hard, get, arm..."
4,on 03/14/13 10:38 AM (PDT),boeserkritiker,2007 Honda Element SUV LX 4dr SUV (2.4L 4cyl 5A),good value,bought vehicle new got k miles served us relia...,4.0,-0.019043,Negative,"[bought, vehicle, new, got, k, miles, served, ..."


In [61]:
df3.to_excel('Sentiment_analysis.xlsx',index=False)